In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()
tf.__version__

'1.13.1'

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [2]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [5]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

<tf.Variable 'Variable:0' shape=(8, 4) dtype=float32, numpy=
array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>


In [6]:
#Weight and bias setting
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.24752675, -1.1883999 ,  3.0299275 ],
       [ 0.22097504, -0.46861383,  0.30399686],
       [-0.4120424 ,  0.54114884,  0.5813529 ],
       [ 0.7729939 , -0.5954734 ,  0.05274786]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 1.4292662 , -0.7926837 ,  0.12507212], dtype=float32)>


# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [7]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis_softmax(tf.Variable(x_data)))

Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor(
[[8.2180515e-02 5.7789753e-04 9.1724157e-01]
 [1.0338405e-03 9.6831718e-06 9.9895644e-01]
 [1.6483627e-04 3.9026993e-08 9.9983513e-01]
 [1.7526309e-06 2.7733249e-10 9.9999821e-01]
 [1.9452028e-02 8.2644772e-07 9.8054713e-01]
 [5.8149721e-02 1.9765312e-05 9.4183058e-01]
 [1.6138338e-02 9.0207260e-07 9.8386073e-01]
 [1.1357455e-02 2.1029420e-07 9.8864228e-01]], shape=(8, 3), dtype=float32)


## 가설을 검증할 Cost 함수를 정의합니다
$$
\begin{align}
cost(h(x),y) & = −log(h(x))  &  if  &  y=1 \\\\\
cost(h(x),y) & = -log(1−h(x))  &  if  &  y=0
\end{align}
$$

### 두 식을 한번에 쓰게되면,

$$
\begin{align}
cost(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [6]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.log(hypothesis) + (1 - labels) * tf.log(1 - hypothesis))
    return cost

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [7]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(hypothesis_softmax(features),labels)))

Iter: 0, Loss: 4.2428
Iter: 100, Loss: 0.9687
Iter: 200, Loss: 0.7369
Iter: 300, Loss: 0.6119
Iter: 400, Loss: 0.4902
Iter: 500, Loss: 0.4068
Iter: 600, Loss: 0.3744
Iter: 700, Loss: 0.3615
Iter: 800, Loss: 0.3540
Iter: 900, Loss: 0.3484
Iter: 1000, Loss: 0.3437
Iter: 1100, Loss: 0.3394
Iter: 1200, Loss: 0.3355
Iter: 1300, Loss: 0.3318
Iter: 1400, Loss: 0.3284
Iter: 1500, Loss: 0.3252
Iter: 1600, Loss: 0.3221
Iter: 1700, Loss: 0.3192
Iter: 1800, Loss: 0.3165
Iter: 1900, Loss: 0.3138
Iter: 2000, Loss: 0.3113
Iter: 2100, Loss: 0.3088
Iter: 2200, Loss: 0.3065
Iter: 2300, Loss: 0.3042
Iter: 2400, Loss: 0.3019
Iter: 2500, Loss: 0.2998
Iter: 2600, Loss: 0.2977
Iter: 2700, Loss: 0.2956
Iter: 2800, Loss: 0.2937
Iter: 2900, Loss: 0.2917
Iter: 3000, Loss: 0.2898
Iter: 3100, Loss: 0.2879
Iter: 3200, Loss: 0.2861
Iter: 3300, Loss: 0.2843
Iter: 3400, Loss: 0.2826
Iter: 3500, Loss: 0.2809
Iter: 3600, Loss: 0.2792
Iter: 3700, Loss: 0.2775
Iter: 3800, Loss: 0.2759
Iter: 3900, Loss: 0.2743
Iter: 4000, 

## Sample 데이터를 넣고 테스트해봅시다.

In [8]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.02830613 0.17317946 0.7985144 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


## 데이터를 이용해서 예측

In [9]:
b = hypothesis_softmax(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data


tf.Tensor(
[[0.00785363 0.02094103 0.9712053 ]
 [0.02830613 0.17317946 0.7985144 ]
 [0.00134477 0.31812218 0.68053305]
 [0.0014618  0.6991368  0.29940143]
 [0.55959857 0.3501925  0.09020896]
 [0.26620394 0.73202544 0.00177065]
 [0.6299294  0.3647595  0.00531116]
 [0.70552933 0.2934914  0.00097937]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 0 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [10]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy
  
test_acc = accuracy_fn(hypothesis_softmax(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.7500
